In [224]:
!pip install puLP

## Programowanie całkowitoliczbowe

# Gdzie zbudować fabrykę?

# Gdzie zbudować magazyn?

Nasza firma ma już fabryki w Łodzi i Olsztynie, może je rozbudować. Chce też wybudować (jeden) magazyn

Decyzje (zmienne decyzyjne)

### Czy budujemy fabrykę w Łodzi?: x1: wartość dodana: 9 milionów, kapitał potrzebny 6 milionów
### W Olsztynie? : x2 5 milionów i 3 miliony
### Magazyn w Łodzi?: 6 milionów i 5 milionów
### W Olsztynie: 4 miliony i 2 miliony

Mamy dostępny kapitał 10 milionów

Ma sens budować w danym mieście mmagazyn tylko jeśli jest tam fabryka

Jak sformułujemy zysk?
Jak ograniczenia


In [192]:
import pulp

from pulp import *

prob = LpProblem("Problem Fabryk", LpMaximize)

In [193]:
#zmienne
x1=LpVariable("Fabryka_Lodz",cat = LpBinary)

x2=LpVariable("Fabryka_Olsztyn",cat = LpBinary)
x3=LpVariable("Magazyn_Lodz",cat = LpBinary)
x4=LpVariable("Magazyn_Olsztyn",cat = LpBinary)


In [194]:
#zysk
prob += 9*x1+5*x2+6*x3+20*x4, "Całkowita wartość dodana"




In [195]:
#ograniczenia

prob += 6*x1+3*x2+5*x3+2*x4<= 10, "dostępny kapitał"


# tylko jeden magazyn ?


# x1 fabryka w Łodzi
# x2 fabryka w Olsztynie
# x3 magazyn w Łodzi  
# x4 magazyn w Olsztynie



In [196]:
prob += x3+x4 <=1, "maksymalnie jeden magazyn"



In [197]:
prob += x1 - x3 >= 0, "zgoda na budowę magazynu w Łodzi tylko jeśli jest fabryka"

prob += x2 - x4 >= 0, "zgoda na budowę magazynu w Olsztynie tylko jeśli jest fabryka"

### gdyby fabryka w Łodzi to w Olsztynie też




In [198]:
prob.solve()

1

In [199]:
def print_rozwiazanie(p):
    p.solve()
    print("Status:", LpStatus[p.status])
    for v in p.variables():
        print (v.name, "=", v.varValue)
    print("Cel = ", value(p.objective))


In [200]:
print_rozwiazanie(prob)

Status: Optimal
Fabryka_Lodz = 0.0
Fabryka_Olsztyn = 1.0
Magazyn_Lodz = 0.0
Magazyn_Olsztyn = 1.0
Cel =  25.0


## Problemy binarne

 - Którą trasę wybrać ( decyzja: wybieram daną trasę)
 - Którą ciężarówkę
 - Czy podjąć inwestycję
    
Decyzje typu tak/nie

## Jesteśmy w stanie za pomocą zmiennych binarnych modelować także bardziej skomplikowane elementy decyzji poprzez tworznie dodatkowych zmiennych decyzyjnych

# Przykład

 Ma być spełniona **jedna** z dwóch nierówności

Zmaksymalizuj
x+y

Dla

0<=x<= 10 0<=y<=10

ale 

x + y <= 3

lub

3y + x <=3




In [201]:
prob = LpProblem("Problem alternatywnych ograniczeń", LpMaximize)
prob

Problem alternatywnych ograniczeń:
MAXIMIZE
None
VARIABLES

In [208]:
import pulp

from pulp import *
#del(prob)
prob2 = LpProblem("Problem alternatywnych ograniczeń", LpMaximize)

x=LpVariable("x",0,10,cat = LpContinuous)

y=LpVariable("y",0,10,cat = LpContinuous)


In [209]:
z=LpVariable("zignoruj_jedno",cat = LpBinary)


In [210]:
prob2 += x+y, "Banalny zysk"

In [211]:
prob2 += x+y <= 3+10000*z, "jedno ograniczenie"

prob2 += 3*x-y <= 3 +10000*(1-z), "drugie ograniczenie"

print_rozwiazanie(prob2)

Status: Optimal
x = 4.3333333
y = 10.0
zignoruj_jedno = 1.0
Cel =  14.3333333


In [212]:
prob2

Problem alternatywnych ograniczeń:
MAXIMIZE
1*x + 1*y + 0
SUBJECT TO
jedno_ograniczenie: x + y - 10000 zignoruj_jedno <= 3

drugie_ograniczenie: 3 x - y + 10000 zignoruj_jedno <= 10003

VARIABLES
x <= 10 Continuous
y <= 10 Continuous
0 <= zignoruj_jedno <= 1 Integer

## Scheduling problem

# Ania Kasia i Piotr mogą dla nas pracować

# Każde z nich może pracować 3 dni w tygodniu

# Każde ma inne stawki

# Ania nie może w pon, Piotr w czwartek i piątek

# stawki za dzień:

#Ania: 150
#Kasia: 160
#Piotr: 140

# We wtorek potrzebujemy 2 osoby, w inne dni jedną

# Funkcja celu?
# Zmienne decyzyjne


In [217]:
prob3 = LpProblem("Problem planu", LpMinimize)




In [218]:
dni_tyg = ["pon","wto","sro","czwar","pio"]

ludzie = ["Ania", "Kasia", "Piotr"]

koszt = [150,160,140]

ludzie_dni = LpVariable.dicts("Plan", (ludzie,dni_tyg), cat="Binary")

print(ludzie_dni)




{'Ania': {'pon': Plan_Ania_pon, 'wto': Plan_Ania_wto, 'sro': Plan_Ania_sro, 'czwar': Plan_Ania_czwar, 'pio': Plan_Ania_pio}, 'Kasia': {'pon': Plan_Kasia_pon, 'wto': Plan_Kasia_wto, 'sro': Plan_Kasia_sro, 'czwar': Plan_Kasia_czwar, 'pio': Plan_Kasia_pio}, 'Piotr': {'pon': Plan_Piotr_pon, 'wto': Plan_Piotr_wto, 'sro': Plan_Piotr_sro, 'czwar': Plan_Piotr_czwar, 'pio': Plan_Piotr_pio}}


In [219]:
prob3 += lpSum(lpSum(koszt*lpSum(ludzie_dni[imie]) for imie,koszt in zip(ludzie,koszt))), "Ile zaplacimy"
print(prob3)

Problem planu:
MINIMIZE
150*Plan_Ania_czwar + 150*Plan_Ania_pio + 150*Plan_Ania_pon + 150*Plan_Ania_sro + 150*Plan_Ania_wto + 160*Plan_Kasia_czwar + 160*Plan_Kasia_pio + 160*Plan_Kasia_pon + 160*Plan_Kasia_sro + 160*Plan_Kasia_wto + 140*Plan_Piotr_czwar + 140*Plan_Piotr_pio + 140*Plan_Piotr_pon + 140*Plan_Piotr_sro + 140*Plan_Piotr_wto + 0
VARIABLES
0 <= Plan_Ania_czwar <= 1 Integer
0 <= Plan_Ania_pio <= 1 Integer
0 <= Plan_Ania_pon <= 1 Integer
0 <= Plan_Ania_sro <= 1 Integer
0 <= Plan_Ania_wto <= 1 Integer
0 <= Plan_Kasia_czwar <= 1 Integer
0 <= Plan_Kasia_pio <= 1 Integer
0 <= Plan_Kasia_pon <= 1 Integer
0 <= Plan_Kasia_sro <= 1 Integer
0 <= Plan_Kasia_wto <= 1 Integer
0 <= Plan_Piotr_czwar <= 1 Integer
0 <= Plan_Piotr_pio <= 1 Integer
0 <= Plan_Piotr_pon <= 1 Integer
0 <= Plan_Piotr_sro <= 1 Integer
0 <= Plan_Piotr_wto <= 1 Integer



In [220]:
ile_potrzebnych_osob = [1,2,1,1,1]
for dzien,potrzebnych_osob in zip(dni_tyg,ile_potrzebnych_osob):

    prob3 += lpSum(ludzie_dni[imie][dzien] for imie in ludzie) == potrzebnych_osob
print(prob3)

Problem planu:
MINIMIZE
150*Plan_Ania_czwar + 150*Plan_Ania_pio + 150*Plan_Ania_pon + 150*Plan_Ania_sro + 150*Plan_Ania_wto + 160*Plan_Kasia_czwar + 160*Plan_Kasia_pio + 160*Plan_Kasia_pon + 160*Plan_Kasia_sro + 160*Plan_Kasia_wto + 140*Plan_Piotr_czwar + 140*Plan_Piotr_pio + 140*Plan_Piotr_pon + 140*Plan_Piotr_sro + 140*Plan_Piotr_wto + 0
SUBJECT TO
_C1: Plan_Ania_pon + Plan_Kasia_pon + Plan_Piotr_pon = 1

_C2: Plan_Ania_wto + Plan_Kasia_wto + Plan_Piotr_wto = 2

_C3: Plan_Ania_sro + Plan_Kasia_sro + Plan_Piotr_sro = 1

_C4: Plan_Ania_czwar + Plan_Kasia_czwar + Plan_Piotr_czwar = 1

_C5: Plan_Ania_pio + Plan_Kasia_pio + Plan_Piotr_pio = 1

VARIABLES
0 <= Plan_Ania_czwar <= 1 Integer
0 <= Plan_Ania_pio <= 1 Integer
0 <= Plan_Ania_pon <= 1 Integer
0 <= Plan_Ania_sro <= 1 Integer
0 <= Plan_Ania_wto <= 1 Integer
0 <= Plan_Kasia_czwar <= 1 Integer
0 <= Plan_Kasia_pio <= 1 Integer
0 <= Plan_Kasia_pon <= 1 Integer
0 <= Plan_Kasia_sro <= 1 Integer
0 <= Plan_Kasia_wto <= 1 Integer
0 <= Plan_Pi

In [221]:
prob3 += ludzie_dni["Kasia"]["pon"] == 0
prob3 += ludzie_dni["Piotr"]["czwar"] == 0
prob3 += ludzie_dni["Piotr"]["pio"] == 0


In [222]:
for imie in ludzie:
    prob3 += lpSum(ludzie_dni[imie]) <= 3



In [183]:
print(prob3)

Problem planu:
MINIMIZE
150*Plan_Ania_czwar + 150*Plan_Ania_pio + 150*Plan_Ania_pon + 150*Plan_Ania_sro + 150*Plan_Ania_wto + 160*Plan_Kasia_czwar + 160*Plan_Kasia_pio + 160*Plan_Kasia_pon + 160*Plan_Kasia_sro + 160*Plan_Kasia_wto + 140*Plan_Piotr_czwar + 140*Plan_Piotr_pio + 140*Plan_Piotr_pon + 140*Plan_Piotr_sro + 140*Plan_Piotr_wto + 0
SUBJECT TO
_C1: Plan_Ania_pon + Plan_Kasia_pon + Plan_Piotr_pon = 1

_C2: Plan_Ania_wto + Plan_Kasia_wto + Plan_Piotr_wto = 2

_C3: Plan_Ania_sro + Plan_Kasia_sro + Plan_Piotr_sro = 1

_C4: Plan_Ania_czwar + Plan_Kasia_czwar + Plan_Piotr_czwar = 1

_C5: Plan_Ania_pio + Plan_Kasia_pio + Plan_Piotr_pio = 1

_C6: Plan_Kasia_pon = 0

_C7: Plan_Piotr_czwar = 0

_C8: Plan_Piotr_pio = 0

_C9: Plan_Ania_czwar + Plan_Ania_pio + Plan_Ania_pon + Plan_Ania_sro
 + Plan_Ania_wto <= 3

_C10: Plan_Kasia_czwar + Plan_Kasia_pio + Plan_Kasia_pon + Plan_Kasia_sro
 + Plan_Kasia_wto <= 3

_C11: Plan_Piotr_czwar + Plan_Piotr_pio + Plan_Piotr_pon + Plan_Piotr_sro
 + Plan_Pi

In [184]:
print_rozwiazanie(prob3)

Status: Optimal
Plan_Ania_czwar = 1.0
Plan_Ania_pio = 1.0
Plan_Ania_pon = 0.0
Plan_Ania_sro = 0.0
Plan_Ania_wto = 1.0
Plan_Kasia_czwar = 0.0
Plan_Kasia_pio = 0.0
Plan_Kasia_pon = 0.0
Plan_Kasia_sro = 0.0
Plan_Kasia_wto = 0.0
Plan_Piotr_czwar = 0.0
Plan_Piotr_pio = 0.0
Plan_Piotr_pon = 1.0
Plan_Piotr_sro = 1.0
Plan_Piotr_wto = 1.0
Cel =  870.0


In [223]:
# Ania Kasia i Piotr mogą dla nas pracować

# Każde z nich może pracować 24h w tygodniu

# Każde ma inne stawki

# Ania nie może w pon, Piotr w czwartek i piątek

# stawki za godzinę:

#Ania: 15
#Kasia: 16
#Piotr: 14

# We wtorek potrzebujemy 12h, w inne dni 8

# Funkcja celu?
# Zmienne decyzyjne
